In [21]:
from elasticsearch import Elasticsearch
import os
import requests

API_KEY = "2oYO3lQwGjIr9RgV+oRa6w==LsM3nPTL7VMFT7m5"
NURT_API_ID = "73bd8bbc"
NURT_API_KEY = "46ff5210638f6e9f5c84ee68eb32033f"

In [2]:
# Get Elasticsearch host from environment variables
elasticsearch_host = "http://localhost:9200"
elasticsearch_username = "elastic"
elasticsearch_password = "p5FE3c=alPhGd20o14bx"
# Connect to Elasticsearch
try:
    es = Elasticsearch(elasticsearch_host, basic_auth=(elasticsearch_username, elasticsearch_password))
    if es.ping():
        print("Successfully connected to Elasticsearch!")
    else:
        print("Failed to connect to Elasticsearch.")
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

Successfully connected to Elasticsearch!


In [9]:
def fetch_recipes(query):
    """Fetch recipes from API-Ninjas and return JSON response."""
    api_url = f"https://api.api-ninjas.com/v1/recipe?query={query}"
    headers = {"X-Api-Key": API_KEY}
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

def store_recipes_in_elasticsearch(recipes):
    """Store the fetched recipes in Elasticsearch."""
    for i, recipe in enumerate(recipes):
        es.index(index="recipes", id=i+1, document=recipe)
    print("Recipes successfully stored in Elasticsearch!")

def search_recipes(keyword):
    """Search for recipes in Elasticsearch."""
    results = es.search(index="recipes", query={
        "match": {
            "title": keyword  # Searches in the recipe title field
        }
    })
    return results["hits"]["hits"]

In [6]:
# Example: Fetch & Store "Italian Wedding Soup"
query = "italian wedding soup"
recipes = fetch_recipes(query)

if recipes:
    store_recipes_in_elasticsearch(recipes)

Recipes successfully stored in Elasticsearch!


In [26]:
# Example search
search_results = search_recipes("soup")

for recipe in search_results:
    print(recipe["_source"])

{'title': 'Italian Wedding Soup', 'ingredients': '1/2 lb Ground beef|1/2 lb Ground veal|1/4 c Italian seasoned bread crumb|1 Egg|1 tb Parsley|Salt and pepper to taste|4 c Chicken broth|2 c Spinach leaves cut into piec|1/4 c Grated Pecorino Romano chees', 'servings': '1 Servings', 'instructions': 'Combine the ground meat, bread crumbs, egg, parsley, salt and pepper in a bowl. Mix well and form into tiny meat balls. Bake on a cookie sheet for 30 minutes at 350F. Meanwhile, bring broth to a boil and add spinach. Cover and boil for 5 minutes. Add the meatballs to the hot broth, bring to a simmer. Stir in the cheese and serve immediately. Rita in Scottsdale 01/02/92 01:41 am'}
{'title': 'Italian Wedding Soup', 'ingredients': '2 qt Chicken stock|1 Chopped carrot|1/2 Chopped onion|1 Chopped celery|2 oz Ground meat; (or ground vegieburger can be used)|1 Egg|1 Sprig of chopped parsely', 'servings': '4 - 6 servin', 'instructions': 'Bring chicken stock to a boil add the chopped carrot,celery and 

In [11]:
def fetch_nutrition(query):
    """Fetch recipes from Nutritionix and return JSON response."""
    api_url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': NURT_API_ID,
        'x-app-key': NURT_API_KEY       
    }
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

In [25]:
#results = fetch_nutrition("apple")
for item in results['common']:
    print(item)

{'common_type': None, 'tag_name': 'apple', 'tag_id': '384', 'food_name': 'apple', 'serving_unit': 'medium (3" dia)', 'photo': {'thumb': 'https://nix-tag-images.s3.amazonaws.com/384_thumb.jpg'}, 'serving_qty': 1, 'locale': 'en_US'}
{'common_type': None, 'tag_name': 'apple', 'tag_id': '384', 'food_name': 'apples', 'serving_unit': 'medium (3" dia)', 'photo': {'thumb': 'https://nix-tag-images.s3.amazonaws.com/384_thumb.jpg'}, 'serving_qty': 1, 'locale': 'en_US'}
{'common_type': None, 'tag_name': 'appletini', 'tag_id': '5031', 'food_name': 'appletini', 'serving_unit': 'cocktail', 'photo': {'thumb': 'https://nix-tag-images.s3.amazonaws.com/5031_thumb.jpg'}, 'serving_qty': 1, 'locale': 'en_US'}
{'common_type': None, 'tag_name': 'Apple pie', 'tag_id': '15060', 'food_name': 'apple pie', 'serving_unit': 'piece (1/8 of 9" dia)', 'photo': {'thumb': 'https://nix-tag-images.s3.amazonaws.com/1291_thumb.jpg'}, 'serving_qty': 1, 'locale': 'en_US'}
{'common_type': None, 'tag_name': 'apple sauce', 'tag_i